# Anscombe quartet (n)

In [129]:
# According to the definition given in Wikipedia, Anscombe’s quartet comprises four datasets 
# that have nearly identical simple statistical properties, yet appear very different when graphed. 
# Each dataset consists of eleven (x,y) points. They were constructed in 1973 by the statistician 
# Francis Anscombe to demonstrate both the importance of graphing data before analyzing it and the effect 
# of outliers on statistical properties.

In [130]:
# Proszę przygotować skrypt (plik z rozszerzeniem *.py), który:

# - będzie uruchamiany z konsoli przez np. python nazwa_skryptu.py
# - musi zawierać funkcję main(), sprawdź tutaj
# - zapisze wyniki obliczeń (mean, std, corr, var) w pliku *.csv
# - wygeneruje jeden obraz jpg z 4 wykresami (subploty, 2 kolumny x 2 wiersze)
# - plik jpg oraz csv mają być zapisane do jednego folderu np. "output"

# Kod musi spełniać zasady:
# - PEP8
# - DRY
# - musi zawierać funkcje z docstring oraz type hints
# - musi być sprawdzony przez inną osobę (zatwierdzony code review)

# Proszę wstawić print screen ze swojego repozytorium z kodem oraz tak aby była widoczna nazwa 
# użytkownika (autora)
# Nazwa pliku (bez polskich znaków):
# Nazwisko_Imie_anscombe_niestacj.jpg

<!-- Proszę przygotować skrypt (plik z rozszerzeniem *.py), który:

- będzie uruchamiany z konsoli przez np. python nazwa_skryptu.py

- musi zawierać funkcję main(), sprawdź tutaj

- wygeneruje jeden obraz jpg z 4 wykresami (subploty, 2 kolumny x 2 wiersze)
    - mean, 
    - standard deviation, 
    - correlation coefficient (Pearson),
    - variance

- zapisze wyniki obliczeń w pliku *.csv

- plik jpg oraz csv mają być zapisane do jednego folderu np. "output"


Kod musi spełniać zasady:

- PEP8

- DRY

- musi zawierać funkcje z docstring oraz type hints

- musi być sprawdzony przez inną osobę (zatwierdzony code review)


Proszę wstawić print screen ze swojego repozytorium z kodem oraz tak aby była widoczna nazwa użytkownika (autora)

Nazwa pliku (bez polskich znaków):

Nazwisko_Imie_anscombe_niestacj.jpg -->

In [131]:
import numpy as np
import pandas as pd
import seaborn as sns
import typing
import matplotlib.pyplot as plt

## data preparation

In [132]:
def data_as_anscombe(multiindex:bool=False) -> anscombe_df:
    """ 
    Makes (multiindex) DataFrame from Anscombe quartet's data.
        Parameters:
            multiindex (bool): Adds multiindex columns to DataFrame. Default = False.
        Returns:
            anscombe_df (DataFrame): Anscombe's quartet as DataFrame.
    """
    
    x = [10, 8, 13, 9, 11, 14, 6, 4, 12, 7, 5]
    y1 = [8.04, 6.95, 7.58, 8.81, 8.33, 9.96, 7.24, 4.26, 10.84, 4.82, 5.68]
    y2 = [9.14, 8.14, 8.74, 8.77, 9.26, 8.10, 6.13, 3.10, 9.13, 7.26, 4.74]
    y3 = [7.46, 6.77, 12.74, 7.11, 7.81, 8.84, 6.08, 5.39, 8.15, 6.42, 5.73]
    x4 = [8, 8, 8, 8, 8, 8, 8, 19, 8, 8, 8]
    y4 = [6.58, 5.76, 7.71, 8.84, 8.47, 7.04, 5.25, 12.50, 5.56, 7.91, 6.89]

    list_of_tuples = list(zip(x, y1, x, y2, x, y3, x4, y4))
    
    anscombe_df = pd.DataFrame(list_of_tuples)
    
    if multiindex == True:
        cols = [np.array(["I", "I", "II", "II", "III", "III", "IV", "IV"]),
            np.array(["x", "y1", "x", "y2", "x", "y3", "x4", "y4"])]
        
        anscombe_df.columns = cols
    
    return anscombe_df

In [133]:
df = data_as_anscombe(multiindex=True)
df

I         II       III         IV       
     x     y1   x    y2   x     y3  x4     y4
0   10   8.04  10  9.14  10   7.46   8   6.58
1    8   6.95   8  8.14   8   6.77   8   5.76
2   13   7.58  13  8.74  13  12.74   8   7.71
3    9   8.81   9  8.77   9   7.11   8   8.84
4   11   8.33  11  9.26  11   7.81   8   8.47
5   14   9.96  14  8.10  14   8.84   8   7.04
6    6   7.24   6  6.13   6   6.08   8   5.25
7    4   4.26   4  3.10   4   5.39  19  12.50
8   12  10.84  12  9.13  12   8.15   8   5.56
9    7   4.82   7  7.26   7   6.42   8   7.91
10   5   5.68   5  4.74   5   5.73   8   6.89

## calculations

In [134]:
def anscombe_statistics():
    """ """
    pass

In [135]:
# mean

In [136]:
# std

In [137]:
# corr

In [138]:
# var

## visualization

In [139]:
# anscombe_df.plot(subplots=True, figsize=(8, 16), style='o', ms=8, grid=True);

In [140]:
# scatter_plot_df = anscombe_df.reset_index()
# fig, axs = plt.subplots(2,2, sharex=True, sharey=True, figsize=(6, 6))
# axs[0, 0].set(xlim=(0, 20), ylim=(2, 14))
# axs[0, 0].set(xticks=(0, 10, 20), yticks=(4, 8, 12))
# axs[0, 0].scatter(x, y1)
# axs[0, 1].scatter(x, y2)
# axs[1, 0].scatter(x, y3)
# axs[1, 1].scatter(x4, y4)
# fig.tight_layout()
# plt.show()

In [141]:
# sns.pairplot(anscombe_df, corner=True)

## as script .py

In [142]:
# shell command convert .ipynb to .py
#!jupyter nbconvert --to script anscombe_quartet.ipynb